In [1]:
import requests
import geopandas as gpd
from shapely.geometry import MultiPolygon
from shapely.ops import unary_union
import osm2geojson  # pip install osm2geojson

# Bounding box for Toulouse
north, south = 43.67795458910826, 43.525655844942015
west, east = 1.2916778177119257, 1.5582214923841673

# Overpass API query
overpass_url = "https://overpass-api.de/api/interpreter"
query = f"""
[out:json][timeout:180];
(
  way["building"]({south},{west},{north},{east});
  relation["building"]({south},{west},{north},{east});
);
out body;
>;
out skel qt;
"""

# Fetch data
print("⏳ Downloading OSM data for Toulouse...")
response = requests.get(overpass_url, params={"data": query})
data = response.json()

# Convert raw OSM to GeoJSON format
print("🔄 Converting to GeoJSON...")
geojson = osm2geojson.json2geojson(data)

# Load into GeoDataFrame
gdf = gpd.GeoDataFrame.from_features(geojson["features"])
gdf.set_crs("EPSG:4326", inplace=True)

# Fix invalid geometries
print("🧼 Cleaning geometries...")
gdf["geometry"] = gdf["geometry"].buffer(0)

# Merge overlapping/touching buildings
print("🔗 Merging all buildings...")
merged = unary_union(gdf["geometry"])

# Wrap as GeoDataFrame
if isinstance(merged, MultiPolygon):
    merged_gdf = gpd.GeoDataFrame(geometry=list(merged.geoms), crs=gdf.crs)
else:
    merged_gdf = gpd.GeoDataFrame(geometry=[merged], crs=gdf.crs)

# Save to GeoPackage
output_file = "toulouse_buildings_merged.gpkg"
merged_gdf.to_file(output_file, driver="GPKG")

print(f"✅ Done! Merged {len(gdf)} buildings into {len(merged_gdf)} merged blocks.")
print(f"📦 Saved to: {output_file}")


⏳ Downloading OSM data for Toulouse...
🔄 Converting to GeoJSON...
🧼 Cleaning geometries...
🔗 Merging all buildings...


/Users/gabeiras3j/opt/anaconda3/envs/cleanproj/lib/python3.10/site-packages/shapely/set_operations.py:543: RuntimeWarning: invalid value encountered in create_collection
  collections = lib.create_collection(


✅ Done! Merged 321242 buildings into 145005 merged blocks.
📦 Saved to: toulouse_buildings_merged.gpkg
